In [ ]:
#Se importan las respectivas librerias a implementar en el desarrollo del modelo
import sys
import os
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import optimizer_v2
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation 
from tensorflow.python.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K

#Evita que la red neuronal sobreentrene y encuentre un numero de entrenamiento optimo para que la red neuronal funcione de la mejor forma
from keras.callbacks import EarlyStopping

In [ ]:
K.clear_session()

data_fit = './data/train'
data_val = './data/val'
data_test = './data/test'

In [ ]:
#Parametros
altura, longitud = 400, 400
batch_size = 32

epochs = 20
pasos = 200
batch_size = 100
pasos_val = 100
filtros_Conv1 = 32
filtros_Conv2 = 64
filtros_Conv3 = 128
filtro1_size = (3,3)
filtro2_size = (2,2)
pool_size = (2,2)
clases = 2
lr = 0.0005

In [ ]:
#Procesamiento de imagenes
fit_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    rescale=1./255
)

imagen_fit = fit_datagen.flow_from_directory(
    data_fit,
    target_size = (altura, longitud),
    batch_size = batch_size,
    class_mode = 'categorical',
    color_mode='grayscale'
)

imagen_val = val_datagen.flow_from_directory(
    data_val,
    target_size = (altura, longitud),
    batch_size = batch_size,
    class_mode = 'categorical',
    color_mode='grayscale'
)

In [ ]:
#Creación de la Red Neuronal Convolucional
def define_modelo():
  cnn = Sequential()

  cnn.add(Convolution2D(filtros_Conv1, filtro1_size, padding='same', input_shape=(400, 400, 1), activation='relu'))
  cnn.add(MaxPooling2D(pool_size=pool_size))

  cnn.add(Convolution2D(filtros_Conv2, filtro2_size, padding='same', activation='relu'))
  cnn.add(MaxPooling2D(pool_size=pool_size))

  cnn.add(Convolution2D(filtros_Conv3, filtro2_size, padding='same', activation='relu'))
  cnn.add(MaxPooling2D(pool_size=pool_size))

  #Deja en una sola dimension la imagen
  cnn.add(Flatten())
  #Conectar las neuronas con las neuronas de la capa anterior
  cnn.add(Dense(255, activation='relu'))
  #Durante el entrenamiento se enciende solo el 50% de las neuronas en cada paso a la capa anterior
  cnn.add(Dropout(0.5))
  #Ayuda a clasificar las imagenes
  cnn.add(Dense(clases, activation='softmax'))

  opt = optimizer_v2.adam.Adam(learning_rate=lr)

  cnn.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

  return cnn

In [ ]:
cnn = define_modelo()